<a name="top"></a>
<div style="width:1000 px">

<div style="float:right; width:98 px; height:98px;">
<img src="https://cdn.miami.edu/_assets-common/images/system/um-logo-gray-bg.png" alt="Miami Logo" style="height: 98px;">
</div>

<div style="float:right; width:98 px; height:98px;">
<img src="https://media.licdn.com/dms/image/C4E0BAQFlOZSAJABP4w/company-logo_200_200/0/1548285168598?e=2147483647&v=beta&t=g4jl8rEhB7HLJuNZhU6OkJWHW4cul_y9Kj_aoD7p0_Y" alt="STI Logo" style="height: 98px;">
</div>


<h1>Compute Harmonics on the UFS S2S</h1>
By: Tyler M. Fenske
    <br>
Last Edited: 2024-02-01
<br>
<br>    
<br>
This notebook applies harmonics analysis to the UFS S2S forecast data as part of the data pre-processing for future forecasting work. <div style="clear:both"></div>
</div>

<hr style="height:2px;">

### Imports & Functions

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
import os
import glob

In [3]:
%run File_concat_mod_functions.ipynb
#include many of the existing functions to handle the NOAA S2S database

### Harmonics Function

In [6]:
def compute_harmonics(yt, k=5, offbyone=False):
    
    '''This function is based off of Section 8.4 (Frequency Domain - 1. Harmonic Analysis)
        from the textbook Statistical Methods in the Atmospheric Sciences by Wilks (2006).
        This function calculates a specified number of harmonics, or fitted cosine curves,
        for a given time series. It functions very similiarly to fourier analysis.
        
        Inputs: 
        yt : a 1-d time series to perform harmonics analysis on
        k  : the number of harmonics to compute
        
        Outputs: 
        yt0  : the sum of the k-harmonics computed
        ybar : the mean of yt
        C    : the coefficient for each harmonic
        w    : the frequency adjustment for each harmonic
        
        KEYWORDS
        offbyone : use if your iterator should start at 1 (Python iterators start at 0)
                   (do not use if unsure; for long time series (100+), it won't matter)'''

    n = yt.shape[-1]                                                                                                          # Make time the last dimension 
                       
    if (k > int(n/2)):                                                                                                        # Ensure the number of harmonics is within the valid range
        k = int(n/2)                                                                                                          # Set k to the maximum valid number of harmonics
                       
    x    = np.linspace(1, n, n) - 1                                                                                           # Create an array of time indices
    ybar = np.nanmean(yt, axis=-1, keepdims=True)                                                                             # Compute the mean of yt along the time dimension
    freq = 2 * np.pi * x / n                                                                                                  # Compute the frequency array
                   
    harmonics_range = np.arange(1, k + 1)                                                                                     # Create an array of harmonic indices
                   
    cos_terms = np.cos(freq * harmonics_range[:, np.newaxis])[np.newaxis, np.newaxis, ...]                                    # Compute cosine terms for harmonics
    sin_terms = np.sin(freq * harmonics_range[:, np.newaxis])[np.newaxis, np.newaxis, ...]                                    # Compute sine terms for harmonics
                       
    A = (2 / n) * np.nansum(yt[:, :, np.newaxis, :] * cos_terms, axis=-1)                                                     # Compute cosine coefficients
    B = (2 / n) * np.nansum(yt[:, :, np.newaxis, :] * sin_terms, axis=-1)                                                     # Compute sine coefficients
                   
    C = np.sqrt(A**2 + B**2)                                                                                                  # Compute the amplitude of the harmonics
                   
    w = np.where(A == 0, np.pi / 2, np.arctan(B / A))                                                                         # Compute the phase angle
    w = np.where(A < 0, w + np.pi, w)                                                                                         # Adjust phase angle for negative cosine coefficients
    w = np.where(w >= 2 * np.pi, w - 2 * np.pi, w)                                                                            # Ensure phase angle is within the range [0, 2π]
                   
    if (offbyone):                                                                                                            # Check for off-by-one error
        w += np.deg2rad(1 / n)                                                                                                # Adjust phase angle to correct off-by-one error
                   
    yt0 = np.repeat(ybar, n, axis=-1)                                                                                         # Initialize the reconstructed time series with the mean
                   
    freq_adjusted = freq[np.newaxis, np.newaxis, :, np.newaxis]                                                               # Adjust frequency array for broadcasting
                   
    harmonics_range = harmonics_range[np.newaxis, np.newaxis, np.newaxis, :]                                                  # Adjust harmonic range array for broadcasting
                   
    yt0 = yt0 + np.sum(C[:, :, np.newaxis, :] * np.cos(harmonics_range * freq_adjusted - w[:, :, np.newaxis, :]), axis=-1)    # Reconstruct the time series using harmonics

    return (yt0, ybar.squeeze(axis=-1), C, w)                                                                                 # Return the reconstructed time series, mean, amplitude, and phase angle

## Application of the Harmonics Function


### UFS S2S

In [7]:
k = 5

ufs_dir  = '../database_files/UFS_S2S/LEAD/'
ufs_base = '_UFS_S2S_FORECAST_'
suffix   = '.nc'

ufs_var_names = [
	'cape','gust','prate','r2','soilw',
    't2m','u10','v10','wdir','wspeed']

In [12]:
for prototype in ['5', '6', '7', '8', 'MPM']:                                                                                                               # Loop through each prototype
    for lead_type in ['day', 'week']:                                                                                                                       # Loop through each lead type
        if lead_type == 'day':                                                                                                                              # Check if lead type is 'day'
            leads = [f'{i:02d}' for i in range(36)]                                                                                                         # Set leads for 'day' type
        if lead_type == 'week':                                                                                                                             # Check if lead type is 'week'
            leads = [f'{i:02d}' for i in range(6)]                                                                                                          # Set leads for 'week' type
                                     
        for stat in ['Abs', 'AVG', 'MIN', 'MAX']:                                                                                                           # Loop through each statistic type
            for var in ufs_var_names:                                                                                                                       # Loop through each variable name in ufs_var_names
                for lead in leads:                                                                                                                          # Loop through each lead
                    if prototype == '5' and lead_type == 'day' and stat == 'Abs' and var in ['cape', 'gust', 'prate', 'r2']:                                # Skip specific conditions
                        continue                         
                    raw_df = xr.open_dataset(f'{ufs_dir}/{prototype}/{lead_type}/{var}{ufs_base}{stat}_lead{lead}{suffix}',                                 # Open and load the dataset for the current variable, prototype, lead type, and lead
                                             decode_times=True)[var].load()                         
                    raw_df = raw_df.sortby('time').squeeze()                                                                                                # Sort the dataset by time and remove any singleton dimensions
                    if var == 'soilw':                                                                                                                      # Check if the variable is 'soilw'
                        raw_df = raw_df.sel(depthBelowLandLayer=0.)                                                                                         # Select data at the surface layer
                         
                    print(f'Applying Harmonics to: {var} {stat}, Prototype {prototype}, Lead {lead_type} {lead}')                                           # Print a message indicating the current variable, statistic, prototype, lead type, and lead being processed
                                             
                    ufunc_output = xr.apply_ufunc(                                                                                                          # Apply the compute_harmonics function using xarray's apply_ufunc
                        compute_harmonics,                                                                                                                  # Function to apply
                        raw_df,                                                                                                                             # Input dataset
                        k,                                                                                                                                  # Number of harmonics
                        False,                                                                                                                              # Off-by-one correction flag
                        input_core_dims=[['valid_time'], [], []],                                                                                           # Input core dimensions
                        output_core_dims=[['valid_time'], [], ['k'], ['k']],                                                                                # Output core dimensions
                        output_dtypes=[float, float, float, float],                                                                                         # Output data types
                        vectorize=False)                                                                                                                    # Do not vectorize the function
                                             
                    climo = ufunc_output[0].transpose('valid_time', 'latitude', 'longitude').rename(var + '_climo')                                         # Extract and rename the climatology component
                    anoms = raw_df - climo                                                                                                                  # Compute anomalies by subtracting climatology from raw data
                    anoms = anoms.rename(var + '_anoms')                                                                                                    # Rename the anomalies component
                                             
                    means  = ufunc_output[1].rename(var + '_means')                                                                                         # Extract and rename the means component
                    consts = ufunc_output[2].rename(var + '_consts')                                                                                        # Extract and rename the constants component
                    phis   = ufunc_output[3].rename(var + '_phis')                                                                                          # Extract and rename the phase angles component
                                             
                    climo_data = xr.merge([climo, means, consts, phis])                                                                                     # Merge climatology components into a single dataset
                         
                    anoms      = anoms.astype('float32')                                                                                                    # Convert anomalies to float32
                    climo_data = climo_data.astype('float32')                                                                                               # Convert climatology data to float32
                    
                    print(f'Complete. Writing full period anom and climo files for {var} {stat}, Prototype {prototype}, Lead {lead_type} {lead}...')        # Print a message indicating completion of processing
                    
                    anoms.to_netcdf(f'{ufs_dir}/Anoms/{prototype}/{lead_type}/{var}{ufs_base}{stat}_lead{lead}{suffix}')                                    # Save anomalies to a NetCDF file
                    climo_data.to_netcdf(f'{ufs_dir}/Climos/{prototype}/{lead_type}/{var}{ufs_base}{stat}_lead{lead}{suffix}')                              # Save climatology data to a NetCDF file


Applying Harmonics to: soilw Abs, Prototype 5, Lead day 00


/tmp/ipykernel_27034/2061358510.py:27: RuntimeWarning: Mean of empty slice
  ybar = np.nanmean(yt, axis=-1, keepdims=True)
/tmp/ipykernel_27034/2061358510.py:41: RuntimeWarning: invalid value encountered in divide
  w = np.where(A == 0, np.pi/2, np.arctan(B/A))


Complete. Writing full period anom and climo files for soilw Abs, Prototype 5, Lead day 00...
Applying Harmonics to: soilw Abs, Prototype 5, Lead day 01


/tmp/ipykernel_27034/2061358510.py:27: RuntimeWarning: Mean of empty slice
  ybar = np.nanmean(yt, axis=-1, keepdims=True)
/tmp/ipykernel_27034/2061358510.py:41: RuntimeWarning: invalid value encountered in divide
  w = np.where(A == 0, np.pi/2, np.arctan(B/A))


Complete. Writing full period anom and climo files for soilw Abs, Prototype 5, Lead day 01...
Applying Harmonics to: soilw Abs, Prototype 5, Lead day 02


...

Complete. Writing full period anom and climo files for soilw MAX, Prototype MPM, Lead week 03...
Applying Harmonics to: soilw MAX, Prototype MPM, Lead week 04


/tmp/ipykernel_27034/2061358510.py:27: RuntimeWarning: Mean of empty slice
  ybar = np.nanmean(yt, axis=-1, keepdims=True)
/tmp/ipykernel_27034/2061358510.py:41: RuntimeWarning: invalid value encountered in divide
  w = np.where(A == 0, np.pi/2, np.arctan(B/A))


Complete. Writing full period anom and climo files for soilw MAX, Prototype MPM, Lead week 04...
Applying Harmonics to: soilw MAX, Prototype MPM, Lead week 05


/tmp/ipykernel_27034/2061358510.py:27: RuntimeWarning: Mean of empty slice
  ybar = np.nanmean(yt, axis=-1, keepdims=True)
/tmp/ipykernel_27034/2061358510.py:41: RuntimeWarning: invalid value encountered in divide
  w = np.where(A == 0, np.pi/2, np.arctan(B/A))


Complete. Writing full period anom and climo files for soilw MAX, Prototype MPM, Lead week 05...
Applying Harmonics to: t2m MAX, Prototype MPM, Lead week 00
Complete. Writing full period anom and climo files for t2m MAX, Prototype MPM, Lead week 00...
Applying Harmonics to: t2m MAX, Prototype MPM, Lead week 01
Complete. Writing full period anom and climo files for t2m MAX, Prototype MPM, Lead week 01...
Applying Harmonics to: t2m MAX, Prototype MPM, Lead week 02
Complete. Writing full period anom and climo files for t2m MAX, Prototype MPM, Lead week 02...
Applying Harmonics to: t2m MAX, Prototype MPM, Lead week 03
Complete. Writing full period anom and climo files for t2m MAX, Prototype MPM, Lead week 03...
Applying Harmonics to: t2m MAX, Prototype MPM, Lead week 04
Complete. Writing full period anom and climo files for t2m MAX, Prototype MPM, Lead week 04...
Applying Harmonics to: t2m MAX, Prototype MPM, Lead week 05
Complete. Writing full period anom and climo files for t2m MAX, Prot